In [1]:
from langchain.chat_models import ChatOpenAI
# from openai import OpenAI
# client = OpenAI()

In [20]:
yes = """
def state_transition(self, state, action):
    \"\"\"
    Applies an action to the current state and returns the new state.

    :param state: A set of predicates representing the current state.
    :param action: A string representing the action to be applied.
    :return: The new state as a set of predicates.
    \"\"\"

    # Split action into words to extract action type and parameters
    words = action.split()
    action_type = words[0]
    params = words[1:]

    new_state = set(state)  # Create a copy of the current state

    if action_type == "pick-up":
        block = params[0]
        new_state.discard(("clear", block))
        new_state.discard(("ontable", block))
        new_state.add(("holding", block))
        print("naniii")
        # egnwgnwognweoginwf37r230'']]]

    elif action_type == "put-down":
        block = params[0]
        new_state.add(("clear", block))
        new_state.add(("ontable", block))
        new_state.discard(("holding", block))

    elif action_type == "stack":
        block1 = params[0]
        block2 = params[1]
        new_state.discard(("clear", block1))
        new_state.add(("clear", block2))
        new_state.discard(("ontable", block1))
        new_state.add(("on", block1, block2))

    elif action_type == "unstack":
        block1 = params[0]
        block2 = params[1]
        new_state.add(("clear", block1))
        new_state.discard(("clear", block2))
        new_state.add(("ontable", block1))
        new_state.discard(("on", block1, block2))

    return new_state
"""

In [3]:
from main import VirtualBlocksWorld

In [6]:
a = VirtualBlocksWorld()
current_state = {
    "ontable a", "ontable b", "clear a", "clear b", "clear c", "ontable c", "handempty"
}

In [21]:
namespace = {}
exec(yes, globals(), namespace)  # Compile the string into a function
new_method = namespace['state_transition']

SyntaxError: unmatched ']' (<string>, line 24)

In [18]:
import types
a.state_transition = types.MethodType(new_method, a)


In [19]:
a.state_transition(current_state, "pick-up a")

naniii


{('holding', 'a'),
 'clear a',
 'clear b',
 'clear c',
 'handempty',
 'ontable a',
 'ontable b',
 'ontable c'}

In [8]:
llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo-1106'
    # model_name='gpt-4-1106-preview'
)

In [10]:
llm.predict("how do i use templates in langchain")

'To use templates in Langchain, you can follow these steps:\n\n1. Log in to your Langchain account and navigate to the "Templates" section.\n\n2. Click on the "Create New Template" button to start creating a new template.\n\n3. Choose the type of template you want to create, such as a document template, email template, or any other type of template that fits your needs.\n\n4. Fill in the necessary details and content for your template, such as the title, description, and the actual content of the template.\n\n5. Once you have created the template, you can then use it in your Langchain account by selecting it when creating new documents, emails, or other types of content.\n\n6. You can also edit and update your templates as needed, and save them for future use.\n\nBy following these steps, you can easily use templates in Langchain to streamline your content creation process and ensure consistency in your communications.'

In [4]:
initial_state = {
    "on-table a", "on-table b", "on-table c",
    "clear a", "clear b", "clear c", "arm-empty"
}

In [2]:
def apply_action(state, action, parameters):
    """
    Applies an action to the current state and returns the new state.

    :param state: A set of predicates representing the current state.
    :param action: A string representing the action to be applied.
    :param parameters: A list of parameters for the action.
    :return: The new state as a set of predicates.
    """
    # Copy the current state to avoid mutating the original
    new_state = state.copy()

    if action == "pickup":
        ob = parameters[0]
        # Check if the block is clear and on the table and the arm is empty
        if {"clear " + ob, "on-table " + ob, "arm-empty"} <= new_state:
            new_state.remove("clear " + ob)
            new_state.remove("on-table " + ob)
            new_state.remove("arm-empty")
            new_state.add("holding " + ob)

    elif action == "putdown":
        ob = parameters[0]
        # Check if holding the object
        if {"holding " + ob} <= new_state:
            new_state.remove("holding " + ob)
            new_state.add("on-table " + ob)
            new_state.add("clear " + ob)
            new_state.add("arm-empty")

    elif action == "stack":
        ob, underob = parameters
        # Check if holding object and under object is clear
        if {"holding " + ob, "clear " + underob} <= new_state:
            new_state.remove("holding " + ob)
            new_state.remove("clear " + underob)
            new_state.add("on " + ob + " " + underob)
            new_state.add("clear " + ob)
            new_state.add("arm-empty")

    elif action == "unstack":
        ob, underob = parameters
        # Check if ob is on underob, and the arm is empty
        if {"on " + ob + " " + underob, "arm-empty"} <= new_state:
            new_state.remove("on " + ob + " " + underob)
            new_state.add("holding " + ob)
            new_state.remove("arm-empty")
            new_state.add("clear " + underob)

    return new_state


In [6]:
apply_action(initial_state, "pickup", ['a'])

{'clear b', 'clear c', 'holding a', 'on-table b', 'on-table c'}